In [1]:
import os, json,time
import pandas as pd
import numpy as np


In [2]:
df=pd.read_json('dataset/jdft_3d-8-18-2021.json') 

In [ ]:
df.loc[df['optb88vdw_bandgap'] <= 0.01, 'optb88vdw_bandgap'] = 0
df.loc[df['optb88vdw_bandgap'] > 0.01 , 'optb88vdw_bandgap'] = 1

In [ ]:
df.loc[df['optb88vdw_bandgap']  == 0, 'optb88vdw_bandgap'].count()

In [ ]:
df.loc[df['optb88vdw_bandgap']  == 0, 'optb88vdw_bandgap'].count()

In [3]:
df.columns

Index(['jid', 'spg_number', 'spg_symbol', 'formula',
       'formation_energy_peratom', 'func', 'optb88vdw_bandgap', 'atoms',
       'slme', 'magmom_oszicar', 'spillage', 'elastic_tensor',
       'effective_masses_300K', 'kpoint_length_unit', 'maxdiff_mesh',
       'maxdiff_bz', 'encut', 'optb88vdw_total_energy', 'epsx', 'epsy', 'epsz',
       'mepsx', 'mepsy', 'mepsz', 'modes', 'magmom_outcar', 'max_efg',
       'avg_elec_mass', 'avg_hole_mass', 'icsd', 'dfpt_piezo_max_eij',
       'dfpt_piezo_max_dij', 'dfpt_piezo_max_dielectric',
       'dfpt_piezo_max_dielectric_electronic',
       'dfpt_piezo_max_dielectric_ionic', 'max_ir_mode', 'min_ir_mode',
       'n-Seebeck', 'p-Seebeck', 'n-powerfact', 'p-powerfact', 'ncond',
       'pcond', 'nkappa', 'pkappa', 'ehull', 'dimensionality', 'efg',
       'xml_data_link', 'typ', 'exfoliation_energy', 'spg', 'crys', 'density',
       'poisson', 'raw_files', 'nat', 'bulk_modulus_kv', 'shear_modulus_gv',
       'mbj_bandgap', 'hse_gap', 'reference'

In [ ]:
df.info()

In [ ]:
data = df.loc[:,['atoms','formation_energy_peratom','optb88vdw_bandgap','optb88vdw_total_energy','ehull','mbj_bandgap']]

In [ ]:
target_name = ['formation_energy_peratom','optb88vdw_bandgap','optb88vdw_total_energy','ehull','mbj_bandgap']

In [ ]:
target = data[target_name]

In [ ]:
target.info()

In [ ]:
ehull = pd.to_numeric(target.loc[:,'ehull'], errors='coerce')
mbj_bandgap = pd.to_numeric(target.loc[:,'mbj_bandgap'], errors='coerce')

In [ ]:
ehull

In [ ]:
target.info()

In [ ]:
target['ehull']=ehull
target['mbj_bandgap']=mbj_bandgap

In [ ]:
target.drop(labels="mbj_bandgap",axis = 1,inplace=True)
target.dropna(inplace=True)

In [ ]:
target.shape

In [ ]:
df = df[['atoms','formation_energy_peratom']].rename(columns={'formation_energy_peratom':'target'})

In [ ]:
import torch
df = torch.load('/hy-tmp/mp_structure_graph_gap_pbe.pt')


In [ ]:
df[0]

In [ ]:
with open('config.json','r',encoding='utf8')as fp:
        config = json.load(fp)
from train import   train
from AttentionAlignn.model import BondAngleGraphAttention
from AttentionAlignn.config import TrainingConfig
train_config = TrainingConfig(**config)



In [ ]:
t1 = time.time()
net = BondAngleGraphAttention(train_config.model)
#训练模型
train(net,df,train_config)
t2 = time.time()
print("Time taken (s):", t2 - t1)

In [ ]:
import pandas as pd

In [ ]:
megnet = pd.read_json('jdft_3d-8-18-2021.json')

In [ ]:
megnet.atoms[0]

In [ ]:
qm9 = pd.read_json('/hy-tmp/qm9_std_jctc.json')

In [ ]:
megnet.info()

In [ ]:
qm9.info()

In [ ]:
megnet.shape

In [ ]:
megnet.head()

In [ ]:
import pandas as pd
import json

In [ ]:
with open('config.json','r',encoding='utf8')as fp:
        config = json.load(fp)

In [ ]:
config

In [ ]:
import pandas as pd
import json



In [ ]:
perovsites = pd.read_json('predicted_mixed_perovskites.json')
perovsites.head()

In [ ]:
from pymatgen.entries.computed_entries import ComputedStructureEntry
import json
import os
data = json.load(open('predicted_mixed_perovskites.json','r')) 
entries = [ComputedStructureEntry.from_dict(i) for i in data]

In [ ]:
import pandas as pd
e_form=[]
atoms=[]
formula=[]
sp =[]
for entry in entries:
    if entry.name == 'BaSrCa2Cu':
        sp.append(entry)
    formula.append(entry.name)
    e_form.append(entry.data['e-form'])
    atoms.append(entry.structure.as_dict())
df = pd.DataFrame({'formula':formula,'atoms':atoms,'e_form':e_form})

In [ ]:
df.formula[df.formula.duplicated()]

In [ ]:
re = df[df.formula=='BaSrCa2Cu']

In [ ]:
r1= re.iloc[0]
r2 = re.iloc[1]

In [ ]:
r1.atoms 

In [ ]:
r2.atoms

In [ ]:
e1=sp[0]
e2=sp[1]

In [ ]:
e1

In [ ]:
e2

In [ ]:
from pymatgen.core.structure import Structure
Structure.from_dict(df.iloc[0].atoms).as_dict()

In [ ]:
from jarvis.core.atoms import Atoms
box = [[2.715, 2.715, 0], [0, 2.715, 2.715], [2.715, 0, 2.715]]
coords = [[0, 0, 0], [0.25, 0.25, 0.25]]
elements = ["Si", "Si"]
Si = Atoms(lattice_mat=box, coords=coords, elements=elements, cartesian=False)

In [ ]:
Si.to_dict()

In [ ]:
fi = entries[0]

In [ ]:
fi

In [ ]:
path = 'predicted_mixed_perovskites'
id_prop=os.path.join(path,'id_prop.csv')
id_df=[]
for entry in entries:
    id_df.append({'file':entry.name+'.cif','e_form':entry.data['e-form']})
    entry.structure.to(path+'/'+entry.name+'.cif')


In [ ]:
pd.DataFrame(id_df).to_csv(id_prop,header=None,index=None)

In [ ]:
import pandas as pd
import os
path = 'predicted_mixed_perovskites'
id_prop=os.path.join(path,'id_prop.csv')
pd.read_csv(id_prop,header=None,index_col=False)

In [ ]:
len(entries)

In [ ]:
len(x)

In [ ]:
x = os.listdir('predicted_mixed_perovskites')
type(x)

In [ ]:
import json, bz2, pickle, gzip as gz
from pymatgen.entries.computed_entries import ComputedStructureEntry

with bz2.open("dcgat_1_000.json.bz2") as fh:
  data = json.loads(fh.read().decode('utf-8'))

entries = [ComputedStructureEntry.from_dict(i) for i in data["entries"][:1000]]

In [ ]:
from pymatgen.core import Structure

# 创建一个示例结构
structure = Structure.from_spacegroup("Fd-3m", 
                                      [[0, 0, 0], [0, 0.25, 0],[0.25, 0, 0]], 
                                      ["Si"], 
                                      [[4, 4, 4]])

# 获取第一个原子的所有邻居
neighbors = structure.get_all_neighbors(3.0)

# 打印邻居原子的索引和位置
for neighbor in neighbors:
    print(neighbor)


In [ ]:
# 获取第一个原子的所有邻居的索引
neighbors_indices = structure.get_all_neighbors(3.0, include_index=True)
print(neighbors_indices)


In [17]:
from collections import OrderedDict

import torch
from torch import nn, optim

from ignite.engine import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.utils import *
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *

# create default evaluator for doctests

def eval_step(engine, batch):
    return batch

default_evaluator = Engine(eval_step)

# create default optimizer for doctests

param_tensor = torch.zeros([1], requires_grad=True)
default_optimizer = torch.optim.SGD([param_tensor], lr=0.1)

# create default trainer for doctests
# as handlers could be attached to the trainer,
# each test must define his own trainer using `.. testsetup:`

def get_default_trainer():

    def train_step(engine, batch):
        return batch

    return Engine(train_step)

# create default model for doctests

default_model = nn.Sequential(OrderedDict([
    ('base', nn.Linear(4, 2)),
    ('fc', nn.Linear(2, 2))
]))

manual_seed(666)

In [18]:
metric = Precision()
weighted_metric = Precision(average='weighted')
two_class_metric = Precision(average=None) # Returns precision for both classes
metric.attach(default_evaluator, "precision")
weighted_metric.attach(default_evaluator, "weighted precision")
two_class_metric.attach(default_evaluator, "both classes precision")
y_true = torch.tensor([1, 0, 1, 1, 0, 1])
# y_pred = torch.tensor([1, 0, 1, 0, 1, 1])
y_pred = torch.tensor([
        [-0.6983, -0.6880],
        [-0.6970, -0.6893],
        [-0.6976, -0.6888],
        [-0.6967, -0.6896],
        [-0.6972, -0.6891],
        [-0.6976, -0.6887],
        ])
state = default_evaluator.run([[y_pred, y_true]])
print(f"Precision: {state.metrics['precision']}")
print(f"Weighted Precision: {state.metrics['weighted precision']}")
print(f"Precision for class 0 and class 1: {state.metrics['both classes precision']}")

Precision: tensor([0.0000, 0.6667], dtype=torch.float64)
Weighted Precision: 0.4444444444444444
Precision for class 0 and class 1: tensor([0.0000, 0.6667], dtype=torch.float64)


In [19]:
for class_id, class_precision in enumerate(state.metrics['precision']):
    print(f"Precision for class {class_id}: {class_precision:.4f}")

Precision for class 0: 0.0000
Precision for class 1: 0.6667


In [5]:
m = nn.LogSoftmax()
input = torch.randn(2)
output = m(input)

In [6]:
output

tensor([-0.0722, -0.4916])

In [7]:
input

tensor([2.5916, 0.4542])